ASAP CRN Unique ID generation - wave 1

# ASAP CRN Unique ID generation - wave 1


Postmortem-derived Brain Sequencing Collection


25 OCT 2023
Andy Henrie


### Dataset ID
- "ASAP_PBMSC" to identify that it is part of the Postmortem-derived Brain Sequencing Collection
- `ASAP_dataset_id`
    - also need to generate a "team_dataset_id" (Add to CDE/DataDictionary). TeamCODE+"one to two word descriptor"

### Team ID
- hardcoded definitions
- `ASAP_team_id`

### Subject ID
- unique for ASAP
- could exist across several Teams / Datasets
- `ASAP_subject_id`

### Sample ID
- unique for each sample
- multiple could derive from same `ASAP_subject_id`.  
    - multiple brain regions from a single team
    - multiple teams from same biobank
    - "other" repeated samples??
- `ASAP_sample_id`
- Unique ASAP_subject_id + "sample repeat number"

## Study ID: Postmortem-derived Brain Sequencing Collection (PMBDS) 
- All ASAP_dataset_id, and ASAP_subject_id here will start with "ASAP_PMBDS_"


###  Issues

- storing "master" IDs for lookup: choosing json to make an easy `dict` mapper, but could make .csv tables 


In [36]:
# conda create -n lw10 python=3.10 notebook ipykernel pip pandas ijson - y && conda activate lw10

In [1]:
import pandas as pd
from pathlib import Path


from asap_ids import (read_meta_table, get_dtypes_dict, STUDY_PREFIX, DATASET_ID, 
                      load_id_mapper, write_id_mapper, generate_asap_sample_ids,
                      generate_asap_subject_ids, process_meta_files)


                       

%load_ext autoreload
%autoreload 2


Load CDE for properly reading the team tables.

In [2]:
CDE_path = Path.cwd() / "ASAP_CDE_v2.csv" 
CDE = pd.read_csv(CDE_path )
# Initialize the data types dictionary
dtypes_dict = get_dtypes_dict(CDE)


## `ASAP_team_id`

On meta-data ingest, add this to:
- STUDY, PROTOCOL

In [3]:
team_names = ["lee", "hafler", "hardy", "jakobsson", "sherzer","sulzer", "voet","wood"]
[x.upper() for x in team_names]



['LEE', 'HAFLER', 'HARDY', 'JAKOBSSON', 'SHERZER', 'SULZER', 'VOET', 'WOOD']

In [4]:
team_codes = ["LEE", "HAF", "HAR", "JAK", "SHE", "SUL", "VOE", "WOO"]




In [5]:
ASAP_team_id = ["TEAM_" + team_name.upper() for team_name in team_names]
ASAP_team_id 

['TEAM_LEE',
 'TEAM_HAFLER',
 'TEAM_HARDY',
 'TEAM_JAKOBSSON',
 'TEAM_SHERZER',
 'TEAM_SULZER',
 'TEAM_VOET',
 'TEAM_WOOD']

## `ASAP_dataset_id`

This compares with the GP2 "study code".

This is done by hand for now. On meta-data ingest, add this (?) to:
- STUDY, PROTOCOL, SAMPLE



Currently we have:
- Team Lee 
- Team Hardy
- Team Hafler



In [6]:
ASAP_dataset_id = DATASET_ID
ASAP_dataset_id


'ASAP_PMBDS'

## `ASAP_subject_id`


### Subject ID
- unique for ASAP
- could exist across several Teams / Datasets
- `ASAP_subject_id`


On meta-data ingest, add this to:
- SUBJECT

"ASAP_XXXXXXX"

Team Lee:  

Team Hardy:

Team Hafler:



We need to define a function that creates the _master_archive_ (if it doesn't exist), and assigns  



## `ASAP_sample_id`

- unique for each sample
- multiple could derive from same `ASAP_subject_id`
- `ASAP_sample_id`
- Unique ASAP_subject_id + "sample repeat number"


On meta-data ingest, add this to:
- SAMPLE

In [7]:
MASTER_SUBJECT_IDs = pd.DataFrame()



In [8]:

data_path = Path.cwd() / "clean/team-Test/v2_20231110"
# make sure cleaned files are correct


SUBJECT = read_meta_table(f"{data_path}/SUBJECT.csv", dtypes_dict)
CLINPATH = read_meta_table(f"{data_path}/CLINPATH.csv", dtypes_dict)
STUDY = read_meta_table(f"{data_path}/STUDY.csv", dtypes_dict)
PROTOCOL = read_meta_table(f"{data_path}/PROTOCOL.csv", dtypes_dict)
SAMPLE = read_meta_table(f"{data_path}/SAMPLE.csv", dtypes_dict)
DATA = read_meta_table(f"{data_path}/SAMPLE.csv", dtypes_dict)


Examples of how to generate the subj_id_mapper  and samp_id_mapper `dict`s

In [9]:



## test with team Lee
subject_mapper_path = Path.cwd() / "ASAP_subj_map2.json"
source_mapper_path = Path.cwd() / "ASAP_source_map2.json"
gp2_mapper_path = Path.cwd() / "ASAP_gp2_map2.json"
sample_mapper_path = Path.cwd() / "ASAP_samp_map2.json"

asapid_mapper = load_id_mapper(sample_mapper_path) # subject id
sourceid_mapper = load_id_mapper(source_mapper_path) # sample id from source
gp2id_mapper = load_id_mapper(subject_mapper_path) # gp2 id
sampleid_mapper = load_id_mapper(sample_mapper_path) # sample id 


# ud_subj_id_mapper, ud_subject_df, n = generate_asap_subject_ids(subj_id_mapper, SUBJECT)
# ud_samp_id_mapper, sample_df = generate_asap_sample_ids(ud_subj_id_mapper, SAMPLE, n, samp_id_mapper)




id_mapper not found at /Users/ergonyc/Projects/ASAP/meta-clean/ASAP_samp_map2.json
id_mapper not found at /Users/ergonyc/Projects/ASAP/meta-clean/ASAP_source_map2.json
id_mapper not found at /Users/ergonyc/Projects/ASAP/meta-clean/ASAP_subj_map2.json
id_mapper not found at /Users/ergonyc/Projects/ASAP/meta-clean/ASAP_samp_map2.json


In [10]:
data_path = Path.cwd() / "clean/team-Hardy/v2_20231109"
# make sure cleaned files are correct

SUBJECT = read_meta_table(f"{data_path}/SUBJECT.csv", dtypes_dict)
subject_df = SUBJECT.copy()



STUDY_PREFIX = "ASAP_PMBDS_"
DATASET_ID = "ASAP_PMBDS"

# master_df = pd.DataFrame.from_dict(asapid_mapper,
#                                     orient='index',
#                                     columns = ['ASAP_subject_id','subject_id', 'source_subject_id','GP2_id']).reset_index().astype(str)
# master_df

In [11]:

def generate_asap_subject_ids2(asapid_mapper:dict,
                             gp2id_mapper:dict,
                             sourceid_mapper:dict, 
                             subject_df:pd.DataFrame) -> tuple[dict,dict,dict,pd.DataFrame,int]:
    """
    generate new unique_ids for new subject_ids in subject_df table, 
    update the id_mapper with the new ids from the data table

    return t
    """

    # extract the max value of the mapper's third (last) section ([2] or [-1]) to get our n
    if bool(asapid_mapper):
        n = max([int(v.split("_")[2]) for v in asapid_mapper.values() if v]) + 1
    else:
        n = 1
    nstart = n

    # ids_df = subject_df[['subject_id','source_subject_id', 'AMPPD_id', 'GP2_id']].copy()
    ids_df = subject_df.copy()

    # might want to use 'source_subject_id' instead of 'subject_id' since we want to find matches across teams
    # shouldn't actually matter but logically cleaner
    uniq_subj = ids_df['subject_id'].unique()
    dupids_mapper = dict(zip(uniq_subj,
                        [num + nstart for num in range(len(uniq_subj))] ))

    n_asap_id_add = 0
    n_gp2_id_add = 0
    n_source_id_add = 0

    df_dup_chunks = []
    id_source = []
    for subj_id, samp_n in dupids_mapper.items():
        df_dups_subset = ids_df[ids_df.subject_id==subj_id].copy()

        # check if gp2_id is known
        # NOTE:  the gp2_id _might_ not be the GP2ID, but instead the GP2sampleID
        #        we might want to check for a trailing _s\d+ and remove it
        #        need to check w/ GP2 team about this.  The RepNo might be sample timepoint... 
        #        and hence be a "subject" in our context
        #    # df['GP2ID'] = df['GP2sampleID'].apply(lambda x: ("_").join(x.split("_")[:-1]))
        #    # df['SampleRepNo'] = df['GP2sampleID'].apply(lambda x: x.split("_")[-1])#.replace("s",""))

        gp2_id = None
        add_gp2_id = False
        # force skipping of null GP2_ids
        if df_dups_subset['GP2_id'].nunique() > 1:
            print(f"subj_id: {subj_id} has multiple gp2_ids: {df_dups_subset['GP2_id'].to_list()}... something is wrong")
            #TODO: log this
        elif not df_dups_subset['GP2_id'].dropna().empty: # we have a valide GP2_id
            gp2_id = df_dups_subset['GP2_id'].values[0] # values because index was not reset

        if gp2_id in set(gp2id_mapper.keys()):
            asap_subj_id_gp2 = gp2id_mapper[gp2_id]
        else:
            add_gp2_id = True
            asap_subj_id_gp2 = None

        # check if source_id is known
        source_id = None
        add_source_id = False
        if df_dups_subset['source_subject_id'].nunique() > 1:
            print(f"subj_id: {subj_id} has multiple source ids: {df_dups_subset['source_subject_id'].to_list()}... something is wrong")
            #TODO: log this
        elif df_dups_subset['source_subject_id'].isnull().any():
            print(f"subj_id: {subj_id} has no source_id... something is wrong")
            #TODO: log this
        else: # we have a valide source_id
            #TODO: check for `source_subject_id` naming collisions with other teams
            #      e.g. check the `biobank_name`
            source_id = df_dups_subset['source_subject_id'].values[0]

        if source_id in set(sourceid_mapper.keys()):
            asap_subj_id_source = sourceid_mapper[source_id]
        else:
            add_source_id = True
            asap_subj_id_source = None

        # TODO: add AMPPD_id test/mapper 

        # check if source_id is known
        add_subj_id = False
        # check if subj_id (subject_id) is known
        if subj_id in set(asapid_mapper.keys()): # duplicate!!
            # TODO: log this
            # TODO: check for `subject_id` naming collisions with other teams
            asap_subj_id = asapid_mapper[subj_id]
        else:
            add_subj_id = True
            asap_subj_id = None

        # TODO:  improve the logic here so gp2 is the default if it exists.?
        #        we need to check the team_id to make sure it's not a naming collision on subject_id
        #        we need to check the biobank_name to make sure it's not a naming collision on source_subject_id

        testset = set((asap_subj_id, asap_subj_id_gp2, asap_subj_id_source))
        if None in testset:
            testset.remove(None)

        # check that asap_subj_id is not disparate between the maps
        if len(testset) > 1:
            print(f"collission between our ids: {(asap_subj_id, asap_subj_id_gp2, asap_subj_id_source)=}")
            print(f"this is BAAAAD. could be a naming collision with another team on `subject_id` ")

        if len(testset) == 0:  # generate a new asap_subj_id
            # print(samp_n)
            asap_subject_id = f"{STUDY_PREFIX}{samp_n:06}"
            # df_dups_subset.insert(0, 'ASAP_subject_id', asap_subject_id, inplace=True)
        else: # testset should have the asap_subj_id
            asap_subject_id = testset.pop() # but where did it come from?
            # print(f"found {subj_id }:{asap_subject_id} in the maps")
        
        src = []
        if add_subj_id:
            asapid_mapper[subj_id] = asap_subject_id
            n_asap_id_add += 1

        if add_gp2_id and gp2_id is not None:
            gp2id_mapper[gp2_id] = asap_subject_id
            n_gp2_id_add += 1

        if add_source_id and source_id is not None:   
            sourceid_mapper[source_id] = asap_subject_id
            n_source_id_add += 1

        
        df_dup_chunks.append(df_dups_subset)
        id_source.append(src)


    df_dups_wids = pd.concat(df_dup_chunks)

    ASAP_subject_id = df_dups_wids['subject_id'].map(asapid_mapper)
    df_dups_wids.insert(0, 'ASAP_subject_id', ASAP_subject_id)

    print(f"added {n_asap_id_add} new asap_subject_ids")
    print(f"added {n_gp2_id_add} new gp2_ids")
    print(f"added {n_source_id_add} new source_ids")

    return asapid_mapper, gp2id_mapper, sourceid_mapper, df_dups_wids, nstart




In [12]:
output = generate_asap_subject_ids(asapid_mapper,
                                    gp2id_mapper,
                                    sourceid_mapper, 
                                    subject_df)
asapid_mapper, gp2id_mapper,sourceid_mapper, subject_df, n = output

added 64 new asap_subject_ids
added 62 new gp2_ids
added 64 new source_ids


In [13]:
subject_df.head()

,ASAP_subject_id,subject_id,source_subject_id,AMPPD_id,GP2_id,biobank_name,organism,sex,age_at_collection,race,...,hx_dementia_mci,hx_melanoma,education_level,smoking_status,smoking_years,APOE_e4_status,cognitive_status,time_from_baseline,primary_diagnosis,primary_diagnosis_text
0,ASAP_PMBDS_000001,babom,P2/14,NaN,MDGAP-QSBB_000096_s1,QSBB_UK,Human,Female,78,NaN,...,Yes,NaN,NaN,NaN,NaN,NaN,NaN,0,Idiopathic PD,clinpath info: PDD | PD (with dementia)
1,ASAP_PMBDS_000002,borah,P4/11,NaN,NaN,QSBB_UK,Human,Male,63,NaN,...,No,NaN,NaN,NaN,NaN,NaN,NaN,0,Idiopathic PD,clinpath info: PD | PD
2,ASAP_PMBDS_000003,bovon,P95/10,NaN,MDGAP-QSBB_000046_s1,QSBB_UK,Human,Male,81,NaN,...,No,NaN,NaN,NaN,NaN,NaN,NaN,0,Idiopathic PD,clinpath info: PD | NA
3,ASAP_PMBDS_000004,davof,P80/11,NaN,MDGAP-QSBB_000070_s1,QSBB_UK,Human,Male,80,NaN,...,No,NaN,NaN,NaN,NaN,NaN,NaN,0,Idiopathic PD,clinpath info: PD | PD
4,ASAP_PMBDS_000005,dudug,P82/10,NaN,MDGAP-QSBB_000592_s1,QSBB_UK,Human,Female,87,NaN,...,No,NaN,NaN,NaN,NaN,NaN,NaN,0,No PD nor other neurological disorder,clinpath info: Control | Control


In [14]:

data_path = Path.cwd() / "clean/team-Hafler/v2_20231109"
# make sure cleaned files are correct


SUBJECT = read_meta_table(f"{data_path}/SUBJECT.csv", dtypes_dict)
subject_df = SUBJECT.copy()

subject_df.head()

,subject_id,source_subject_id,AMPPD_id,GP2_id,biobank_name,organism,sex,age_at_collection,race,ethnicity,...,hx_dementia_mci,hx_melanoma,education_level,smoking_status,smoking_years,APOE_e4_status,cognitive_status,time_from_baseline,primary_diagnosis,primary_diagnosis_text
0,HC01,hSDG07,NaN,NaN,NKI/NYUGSOM,Human,Female,81,Black or African American,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthy Control,NaN
1,HC02,hSDG13,NaN,NaN,NKI/NYUGSOM,Human,Female,65,White,W,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthy Control,NaN
2,HC03,hSDG101,NaN,NaN,NKI/NYUGSOM,Human,Female,72,White,W,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthy Control,NaN
3,HC04,hSDG10,NaN,NaN,NKI/NYUGSOM,Human,Male,70,White,W,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthy Control,NaN
4,HC05,hSDG30,NaN,NaN,NKI/NYUGSOM,Human,Male,63,Black or African American,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthy Control,NaN


In [15]:
output = generate_asap_subject_ids(asapid_mapper,
                                    gp2id_mapper,
                                    sourceid_mapper, 
                                    subject_df)
asapid_mapper, gp2id_mapper,sourceid_mapper, subject_df, n = output

added 12 new asap_subject_ids
added 0 new gp2_ids
added 12 new source_ids


In [16]:
subject_df.head()

,ASAP_subject_id,subject_id,source_subject_id,AMPPD_id,GP2_id,biobank_name,organism,sex,age_at_collection,race,...,hx_dementia_mci,hx_melanoma,education_level,smoking_status,smoking_years,APOE_e4_status,cognitive_status,time_from_baseline,primary_diagnosis,primary_diagnosis_text
0,ASAP_PMBDS_000065,HC01,hSDG07,NaN,NaN,NKI/NYUGSOM,Human,Female,81,Black or African American,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthy Control,NaN
1,ASAP_PMBDS_000066,HC02,hSDG13,NaN,NaN,NKI/NYUGSOM,Human,Female,65,White,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthy Control,NaN
2,ASAP_PMBDS_000067,HC03,hSDG101,NaN,NaN,NKI/NYUGSOM,Human,Female,72,White,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthy Control,NaN
3,ASAP_PMBDS_000068,HC04,hSDG10,NaN,NaN,NKI/NYUGSOM,Human,Male,70,White,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthy Control,NaN
4,ASAP_PMBDS_000069,HC05,hSDG30,NaN,NaN,NKI/NYUGSOM,Human,Male,63,Black or African American,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthy Control,NaN


In [17]:

data_path = Path.cwd() / "clean/team-Test/v2_20231110"
# make sure cleaned files are correct


SUBJECT = read_meta_table(f"{data_path}/SUBJECT.csv", dtypes_dict)
subject_df = SUBJECT.copy()

subject_df.head()

,subject_id,source_subject_id,AMPPD_id,GP2_id,biobank_name,organism,sex,age_at_collection,race,ethnicity,...,hx_dementia_mci,hx_melanoma,education_level,smoking_status,smoking_years,APOE_e4_status,cognitive_status,time_from_baseline,primary_diagnosis,primary_diagnosis_text
0,HC_1225,12-25,NaN,NaN,Banner Sun Health Research Institute,Human,Male,80,White,Not Reported,...,No,NaN,NaN,Unknown,NaN,33,Normal,0,No PD nor other neurological disorder,NaN
1,HC_0602,06-02,NaN,NaN,Banner Sun Health Research Institute,Human,Male,84,White,Not Reported,...,Yes,NaN,NaN,Unknown,NaN,34,Normal,0,Other neurological disorder,Mild Cognitive Impairment
2,PD_0009,00-09,NaN,NaN,Banner Sun Health Research Institute,Human,Male,64,White,Not Reported,...,Yes,NaN,NaN,Unknown,NaN,33,MCI,0,Idiopathic PD,NaN
3,PD_1921,19-21,NaN,NaN,Banner Sun Health Research Institute,Human,Male,82,White,Not Reported,...,Yes,NaN,NaN,Unknown,NaN,33,MCI,0,Idiopathic PD,NaN
4,PD_2058,20-58,NaN,NaN,Banner Sun Health Research Institute,Human,Male,87,White,Not Reported,...,Yes,NaN,NaN,Unknown,NaN,34,Normal,0,Idiopathic PD,NaN


In [18]:
output = generate_asap_subject_ids(asapid_mapper,
                                    gp2id_mapper,
                                    sourceid_mapper, 
                                    subject_df)
asapid_mapper, gp2id_mapper,sourceid_mapper, subject_df, n = output

added 45 new asap_subject_ids
added 0 new gp2_ids
added 35 new source_ids


In [395]:
subject_df

,ASAP_subject_id,subject_id,source_subject_id,AMPPD_id,GP2_id,biobank_name,organism,sex,age_at_collection,race,...,hx_dementia_mci,hx_melanoma,education_level,smoking_status,smoking_years,APOE_e4_status,cognitive_status,time_from_baseline,primary_diagnosis,primary_diagnosis_text
0,ASAP_PMBDS_000077,HC_1225,12-25,NaN,NaN,Banner Sun Health Research Institute,Human,Male,80,White,...,No,NaN,NaN,Unknown,NaN,33,Normal,0,No PD nor other neurological disorder,NaN
1,ASAP_PMBDS_000078,HC_0602,06-02,NaN,NaN,Banner Sun Health Research Institute,Human,Male,84,White,...,Yes,NaN,NaN,Unknown,NaN,34,Normal,0,Other neurological disorder,Mild Cognitive Impairment
2,ASAP_PMBDS_000079,PD_0009,00-09,NaN,NaN,Banner Sun Health Research Institute,Human,Male,64,White,...,Yes,NaN,NaN,Unknown,NaN,33,MCI,0,Idiopathic PD,NaN
3,ASAP_PMBDS_000080,PD_1921,19-21,NaN,NaN,Banner Sun Health Research Institute,Human,Male,82,White,...,Yes,NaN,NaN,Unknown,NaN,33,MCI,0,Idiopathic PD,NaN
4,ASAP_PMBDS_000081,PD_2058,20-58,NaN,NaN,Banner Sun Health Research Institute,Human,Male,87,White,...,Yes,NaN,NaN,Unknown,NaN,34,Normal,0,Idiopathic PD,NaN
5,ASAP_PMBDS_000082,PD_1441,14-41,NaN,NaN,Banner Sun Health Research Institute,Human,Female,69,White,...,No,NaN,NaN,Unknown,NaN,33,Normal,0,Idiopathic PD,NaN
6,ASAP_PMBDS_000083,PD_1344,13-44,NaN,NaN,Banner Sun Health Research Institute,Human,Female,88,White,...,No,NaN,NaN,Unknown,NaN,33,Normal,0,Idiopathic PD,NaN
7,ASAP_PMBDS_000084,HC_1939,19-39,NaN,NaN,Banner Sun Health Research Institute,Human,Female,93,White,...,No,NaN,NaN,Unknown,NaN,33,Normal,0,No PD nor other neurological disorder,NaN
8,ASAP_PMBDS_000085,HC_1308,13-08,NaN,NaN,Banner Sun Health Research Institute,Human,Male,87,White,...,No,NaN,NaN,Unknown,NaN,33,MCI,0,No PD nor other neurological disorder,NaN
9,ASAP_PMBDS_000086,HC_1862,18-62,NaN,NaN,Banner Sun Health Research Institute,Human,Male,80,White,...,No,NaN,NaN,Unknown,NaN,33,Normal,0,No PD nor other neurological disorder,NaN


Try to deal with each subj_id individually so nodups and dups get sensible mappings.

I think this WORKS!!!



In [19]:

def generate_asap_sample_ids3(asapid_mapper:dict, 
                             sample_df:pd.DataFrame, 
                             sampleid_mapper:dict) -> tuple[dict, pd.DataFrame]:
    """
    generate new unique_ids for new sample_ids in sample_df table, 
    update the id_mapper with the new ids from the data table

    return the updated id_mapper and updated sample_df
    """

    ud_sampleid_mapper = sampleid_mapper.copy()
    # 
    uniq_subj = sample_df.subject_id.unique()
    # check for subject_id collisions in the sampleid_mapper
    if intersec := set(uniq_subj) & set(ud_sampleid_mapper.values()): 
        print(f"found {len(intersec)} subject_id collisions in the sampleid_mapper")
        
    df_chunks = []
    for subj_id in uniq_subj:

        df_subset = sample_df[sample_df.subject_id==subj_id].copy()
        asap_id = asapid_mapper[subj_id]

        dups = df_subset[df_subset.duplicated(keep=False, subset=['sample_id'])].sort_values('sample_id').reset_index(drop = True).copy()
        nodups = df_subset[~df_subset.duplicated(keep=False, subset=['sample_id'])].sort_values('sample_id').reset_index(drop = True).copy()
   
        asap_id = asapid_mapper[subj_id]
        if bool(ud_sampleid_mapper):
            sns = [get_samp(v) for v in ud_sampleid_mapper.values() if get_id(v)==asap_id]
            n = max(sns) + 1  
            print(n)
            # if len(sns) > 0: n = max(sns) + 1            
            # else: n = 1; print("why we buggin?")
        else:
            n = 1

        if nodups.shape[0]>0:
            # ASSIGN IDS
            asap_nodups = [f'{asap_id}_s{i+n:03}' for i in range(nodups.shape[0])]
            # nodups['ASAP_sample_id'] = asap_nodups
            nodups.loc[:, 'ASAP_sample_id'] = asap_nodups
            n = n + nodups.shape[0]
            samples_nodups = nodups['sample_id'].unique()

            nodup_mapper = dict(zip(nodups['sample_id'],asap_nodups))
            # df_subset['samp_rep_no'] = [f'r{i+1:02}' for i in range(df_subset.shape[0])]
            # # make a new column with the asap_sample_id
            # # and insert it at the beginning of the dataframe
            # df_subset['ASAP_sample_id'] = df_subset['asap_sample'] + '_' + df_subset['samp_rep_no']

            df_chunks.append(nodups)

        if dups.shape[0]>0:
            for dup_id in dups['sample_id'].unique():
                # first peel of any sample_ids that were already named in nodups, 
                if dup_id in samples_nodups:
                    asap_dup = nodup_mapper[dup_id]                    
                else:
                    # then assign ids to the rest.
                    asap_dup = f'{asap_id}_s{n:03}'
                    dups.loc[dups.sample_id==dup_id, 'ASAP_sample_id'] = asap_dup
                    n += 1
            df_chunks.append(dups)


    df_wids = pd.concat(df_chunks)
    id_mapper = dict(zip(df_wids['sample_id'],
                        df_wids['ASAP_sample_id']))

    ud_sampleid_mapper.update(id_mapper)


    out_df = sample_df.copy()
    ASAP_sample_id = out_df['sample_id'].map(ud_sampleid_mapper)
    out_df.insert(0, 'ASAP_sample_id', ASAP_sample_id)

    print(ud_sampleid_mapper)
    return ud_sampleid_mapper, out_df

In [21]:

def get_samp(v):
    return int(v.split("_")[3].replace("s","")) 

def get_id(v):
    return v[:17] 

def get_repn(v):
    return int(v.split("_")[4].replace("r","")) 

sample_df = SAMPLE.copy()

ud_sampleid_mapper, ud_sample_df = generate_asap_sample_ids(asapid_mapper, sample_df, sampleid_mapper)


KeyError: nan

In [51]:
ud_sample_df

,ASAP_sample_id,sample_id,subject_id,source_sample_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id
0,ASAP_PMBDS_000001_s001,babom_ACG,babom,NaN,NaN,1,0,BATCH_2,brain,ACG,...,PATO:0000383 (female),NaN,MONDO:0005180,UBERON:0009835,NaN,EFO:0008913,nucleus,NaN,NaN,NaN
1,ASAP_PMBDS_000001_s002,babom_IPL,babom,NaN,NaN,1,0,BATCH_2,brain,IPL,...,PATO:0000383 (female),NaN,MONDO:0005180,UBERON:0006088,NaN,EFO:0008913,nucleus,NaN,NaN,NaN
2,ASAP_PMBDS_000002_s001,borah_ACG,borah,NaN,NaN,1,0,BATCH_2,brain,ACG,...,PATO:0000384 (male),NaN,MONDO:0005180,UBERON:0009835,NaN,EFO:0008913,nucleus,NaN,NaN,NaN
3,ASAP_PMBDS_000002_s002,borah_IPL,borah,NaN,rep1,2,1,BATCH_2,brain,IPL,...,PATO:0000384 (male),NaN,MONDO:0005180,UBERON:0006088,NaN,EFO:0008913,nucleus,NaN,NaN,NaN
4,ASAP_PMBDS_000002_s002,borah_IPL,borah,NaN,rep2,2,1,BATCH_5,brain,IPL,...,PATO:0000384 (male),NaN,MONDO:0005180,UBERON:0006088,NaN,EFO:0008913,nucleus,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,ASAP_PMBDS_000063_s002,zidip_IPL,zidip,NaN,NaN,1,0,BATCH_4,brain,IPL,...,PATO:0000383 (female),NaN,MONDO:0005180,UBERON:0006088,NaN,EFO:0008913,nucleus,NaN,NaN,NaN
134,ASAP_PMBDS_000064_s001,zupam_ACG,zupam,NaN,rep1,2,1,BATCH_2,brain,ACG,...,PATO:0000383 (female),NaN,MONDO:0005180,UBERON:0009835,NaN,EFO:0008913,nucleus,NaN,NaN,NaN
135,ASAP_PMBDS_000064_s001,zupam_ACG,zupam,NaN,rep2,2,1,BATCH_5,brain,ACG,...,PATO:0000383 (female),NaN,MONDO:0005180,UBERON:0009835,NaN,EFO:0008913,nucleus,NaN,NaN,NaN
136,ASAP_PMBDS_000064_s002,zupam_IPL,zupam,NaN,rep1,2,1,BATCH_2,brain,IPL,...,PATO:0000383 (female),NaN,MONDO:0005180,UBERON:0006088,NaN,EFO:0008913,nucleus,NaN,NaN,NaN


In [90]:

# new entries
newids_df = ids_df[ids_df['ASAP_subject_id'].isnull()].reset_index(drop = True).copy()

# entries with existing ids
haveids_df = ids_df[~ids_df['ASAP_subject_id'].isnull()].reset_index(drop = True).copy()

In [91]:
data_duplicated = pd.merge(subject_df, master_df, on=['subject_id'], how='inner')

assert data_duplicated.empty, "There are duplicated subject_ids in the data table"

In [92]:
newids_df.empty

False

In [93]:
# newids_df is not Empty # i.e. we have new ids to add

df_dups = ids_df[ids_df.duplicated(keep=False, subset=['subject_id'])].sort_values('subject_id').reset_index(drop = True).copy()
df_nodups = ids_df[~ids_df.duplicated(keep=False, subset=['subject_id'])].sort_values('subject_id').reset_index(drop = True).copy()


# TODO: CODE THE UNPACKER OF THE ASAP_sample_id to deal with duplicates after creating a duplicate
# # unpack the old entries
# df_wids = df_subset[~df_subset['GP2sampleID'].isnull()].reset_index(drop = True).copy()
# df_wids['GP2ID'] = df_wids['GP2sampleID'].apply(lambda x: ("_").join(x.split("_")[:-1]))
# df_wids['SampleRepNo'] = df_wids['GP2sampleID'].apply(lambda x: x.split("_")[-1])#.replace("s",""))

df_dups.shape, df_nodups.shape

((26, 27), (13, 27))

In [ ]:


dups_df = ids_df[ids_df.duplicated(keep=False, subset=['subject_id'])].sort_values('subject_id').reset_index(drop = True).copy()
if dups_df.shape[0]>0:
    dupids_mapper = dict(zip(dups_df.subject_id.unique(),
                        [num+n for num in range(len(dups_df.subject_id.unique()))]))
    
    dup_df_chunks = []
    for clin_id, id in dupids_mapper.items():
        dups_subset = df_dups[df_dups.clinical_id==clin_id].copy()
        dups_subset['GP2ID'] = [f'{study_code}_{id:06}' for i in range(dups_subset.shape[0])]
        dups_subset['SampleRepNo'] = ['s'+str(i+1) for i in range(dups_subset.shape[0])]
        dups_subset['GP2sampleID'] = dups_subset['GP2ID'] + '_' + dups_subset['SampleRepNo']
        dup_df_chunks.append(df_dups_subset)
    df_dups_wids = pd.concat(df_dup_chunks)

df_nodups = dfproc[~dfproc.duplicated(keep=False, subset=['clinical_id'])].sort_values('clinical_id').reset_index(drop = True).copy()

if df_dups.shape[0]>0:
    n =  len(list(dupids_mapper.values())) + n
else:
    n = n

uids = [str(id) for id in df_nodups['sample_id'].unique()]
mapid = {}
for uid in uids:
    mapid[uid]= n
    n += 1
df_nodups_wids = df_nodups.copy()
df_nodups_wids['uid_idx'] = df_nodups_wids['sample_id'].map(mapid)
df_nodups_wids['GP2ID'] = [f'{study_code}_{i:06}' for i in df_nodups_wids.uid_idx]
df_nodups_wids['uid_idx_cumcount'] = df_nodups_wids.groupby('GP2ID').cumcount() + 1
df_nodups_wids['GP2sampleID'] = df_nodups_wids.GP2ID + '_s' + df_nodups_wids.uid_idx_cumcount.astype('str')
df_nodups_wids['SampleRepNo'] = 's' + df_nodups_wids.uid_idx_cumcount.astype('str')
df_nodups_wids.drop(['uid_idx','uid_idx_cumcount'], axis = 1, inplace = True)

if df_dups.shape[0]>0:
    df_newids = pd.concat([df_dups_wids, df_nodups_wids])
else:
    df_newids = df_nodups_wids


In [ ]:

# make new ids for new entries
n=int(max(study_tracker_df['master_GP2sampleID'].to_list()).split("_")[1])+1
df_newids = generategp2ids.getgp2idsv2(df_newids, n, study)
df_subset = pd.concat([df_newids, df_wids], axis = 0)
study_subsets.append(df_subset)
log_new.append(df_newids[['study','clinical_id','sample_id','GP2sampleID']])



In [ ]:
    
# newids_df is  Empty # i.e. we ONLY new ids to add
# TO CONSIDER THE CASE IN WHICH WE ONLY HAD DUPLICATE IDS MAPPED ON THE MASTER FILE

# just unpack the entries
df_subset['GP2ID'] = df_subset['GP2sampleID'].apply(lambda x: ("_").join(x.split("_")[:-1]))
df_subset['SampleRepNo'] = df_subset['GP2sampleID'].apply(lambda x: x.split("_")[-1])#.replace("s",""))
study_subsets.append(df_subset)



In [ ]:

                # Brand new data - NO STUDY TRACKER FOR THIS COHORT
                else:
                    study = study
                    new_clinicaldups = False # Duplicates from master key json are treated differently to brand new data
                    n = 1
                    df_newids = generategp2ids.getgp2idsv2(df_subset, n, study)
                    study_subsets.append(df_newids)












df_dups = dfproc[dfproc.duplicated(keep=False, subset=['clinical_id'])].sort_values('clinical_id').reset_index(drop = True).copy()
if df_dups.shape[0]>0:
    dupids_mapper = dict(zip(df_dups.clinical_id.unique(),
                        [num+n for num in range(len(df_dups.clinical_id.unique()))]))
    
    df_dup_chunks = []
    for clin_id, gp2id in dupids_mapper.items():
        df_dups_subset = df_dups[df_dups.clinical_id==clin_id].copy()
        df_dups_subset['GP2ID'] = [f'{study_code}_{gp2id:06}' for i in range(df_dups_subset.shape[0])]
        df_dups_subset['SampleRepNo'] = ['s'+str(i+1) for i in range(df_dups_subset.shape[0])]
        df_dups_subset['GP2sampleID'] = df_dups_subset['GP2ID'] + '_' + df_dups_subset['SampleRepNo']
        df_dup_chunks.append(df_dups_subset)
    df_dups_wids = pd.concat(df_dup_chunks)

df_nodups = dfproc[~dfproc.duplicated(keep=False, subset=['clinical_id'])].sort_values('clinical_id').reset_index(drop = True).copy()

if df_dups.shape[0]>0:
    n =  len(list(dupids_mapper.values())) + n
else:
    n = n

uids = [str(id) for id in df_nodups['sample_id'].unique()]
mapid = {}
for uid in uids:
    mapid[uid]= n
    n += 1
df_nodups_wids = df_nodups.copy()
df_nodups_wids['uid_idx'] = df_nodups_wids['sample_id'].map(mapid)
df_nodups_wids['GP2ID'] = [f'{study_code}_{i:06}' for i in df_nodups_wids.uid_idx]
df_nodups_wids['uid_idx_cumcount'] = df_nodups_wids.groupby('GP2ID').cumcount() + 1
df_nodups_wids['GP2sampleID'] = df_nodups_wids.GP2ID + '_s' + df_nodups_wids.uid_idx_cumcount.astype('str')
df_nodups_wids['SampleRepNo'] = 's' + df_nodups_wids.uid_idx_cumcount.astype('str')
df_nodups_wids.drop(['uid_idx','uid_idx_cumcount'], axis = 1, inplace = True)

if df_dups.shape[0]>0:
    df_newids = pd.concat([df_dups_wids, df_nodups_wids])
else:
    df_newids = df_nodups_wids








# might want to use 'source_subject_id' instead of 'subject_id' since we want to find matches across teams
# shouldn't actually matter but logically cleaner
uids = [str(id) for id in df_nodups_wids['subject_id'].unique()]
mapid = {}
for uid in uids:
    mapid[uid]= n
    n += 1

    df_nodups_wids['uid_idx'] = df_nodups_wids['subject_id'].map(mapid)
    # make a new column with the ASAP_subject_id
    # and insert it at the beginning of the dataframe
    ASAP_subject_id = [f'{STUDY_PREFIX}{i:06}' for i in df_nodups_wids.uid_idx]
    df_nodups_wids.insert(0, 'ASAP_subject_id', ASAP_subject_id)

    df_nodups_wids['uid_idx_cumcount'] = df_nodups_wids.groupby('ASAP_subject_id').cumcount() + 1
    asap_id_mapper = dict(zip(df_nodups_wids['subject_id'], df_nodups_wids['ASAP_subject_id']))

    # update the subj_id_mapper
    subj_id_mapper.update(asap_id_mapper)

In [69]:
# get the last incrimented id from master_df
if newids_df.empty:# TO CONSIDER THE CASE IN WHICH WE ONLY HAD DUPLICATE IDS MAPPED ON THE MASTER FILE
    # just unpack the entries





else:   # Get new  IDs
    if master_df.empty:
        n = 1
    else:
        n = int(max(master_df['ASAP_subject_id'].to_list()).split("_")[2])+1
    nstart = n
    



In [ ]:
# GET GP2 IDs METADATA for new CLINICAL-SAMPLE ID pairs
df_newids = df_subset[df_subset['GP2sampleID'].isnull()].reset_index(drop = True).copy()
if not df_newids.empty: # Get new GP2 IDs
    # unpack the entries
    df_wids = df_subset[~df_subset['GP2sampleID'].isnull()].reset_index(drop = True).copy()
    df_wids['GP2ID'] = df_wids['GP2sampleID'].apply(lambda x: ("_").join(x.split("_")[:-1]))
    df_wids['SampleRepNo'] = df_wids['GP2sampleID'].apply(lambda x: x.split("_")[-1])#.replace("s",""))

    # make new ids for new entries
    n=int(max(study_tracker_df['master_GP2sampleID'].to_list()).split("_")[1])+1
    df_newids = generategp2ids.getgp2idsv2(df_newids, n, study)
    df_subset = pd.concat([df_newids, df_wids], axis = 0)
    study_subsets.append(df_subset)
    log_new.append(df_newids[['study','clinical_id','sample_id','GP2sampleID']])
    
else: # TO CONSIDER THE CASE IN WHICH WE ONLY HAD DUPLICATE IDS MAPPED ON THE MASTER FILE
    # just unpack the entries
    df_subset['GP2ID'] = df_subset['GP2sampleID'].apply(lambda x: ("_").join(x.split("_")[:-1]))
    df_subset['SampleRepNo'] = df_subset['GP2sampleID'].apply(lambda x: x.split("_")[-1])#.replace("s",""))
    study_subsets.append(df_subset)

In [ ]:


df_nodups_wids = subject_df.copy()
# might want to use 'source_subject_id' instead of 'subject_id' since we want to find matches across teams
# shouldn't actually matter but logically cleaner
uids = [str(id) for id in df_nodups_wids['subject_id'].unique()]
mapid = {}
for uid in uids:
    mapid[uid]= n
    n += 1

df_nodups_wids['uid_idx'] = df_nodups_wids['subject_id'].map(mapid)
# make a new column with the ASAP_subject_id
# and insert it at the beginning of the dataframe
ASAP_subject_id = [f'{STUDY_PREFIX}{i:06}' for i in df_nodups_wids.uid_idx]
df_nodups_wids.insert(0, 'ASAP_subject_id', ASAP_subject_id)

df_nodups_wids['uid_idx_cumcount'] = df_nodups_wids.groupby('ASAP_subject_id').cumcount() + 1
asap_id_mapper = dict(zip(df_nodups_wids['subject_id'], df_nodups_wids['ASAP_subject_id']))

# update the subj_id_mapper
subj_id_mapper.update(asap_id_mapper)

In [ ]:
if not df_newids.empty: # Get new GP2 IDs
    df_wids = df_subset[~df_subset['ASAP_subject_id'].isnull()].reset_index(drop = True).copy()
    df_wids['asap_id'] = df_wids['ASAP_subject_id'].apply(lambda x: ("_").join(x.split("_")[:-1]))
    df_wids['samp_rep_no'] = df_wids['GP2samasap_idpleID'].apply(lambda x: x.split("_")[-1])#.replace("s",""))

    n=int(max(study_tracker_df['master_GP2sampleID'].to_list()).split("_")[1])+1
    df_newids = generategp2ids.getgp2idsv2(df_newids, n, study)
    df_subset = pd.concat([df_newids, df_wids], axis = 0)
    study_subsets.append(df_subset)
    log_new.append(df_newids[['study','clinical_id','sample_id','GP2sampleID']])
    
else: # TO CONSIDER THE CASE IN WHICH WE ONLY HAD DUPLICATE IDS MAPPED ON THE MASTER FILE

    
    df_subset['GP2ID'] = df_subset['GP2sampleID'].apply(lambda x: ("_").join(x.split("_")[:-1]))
    df_subset['SampleRepNo'] = df_subset['GP2sampleID'].apply(lambda x: x.split("_")[-1])#.replace("s",""))
    study_subsets.append(df_subset)

In [ ]:

# check for duplicates in the subject_df
df_dups = master_df[master_df.duplicated(keep=False, subset=['subject_id'])].sort_values('subject_id').reset_index(drop = True).copy()



# if there are duplicates, then generate new ids for the duplicates
if not df_dups.empty:
    # generate new ids for the duplicates
    new_ids = generate_new_ids(df_dups, 'subject_id')
    # update the master_df with the new ids
    master_df = pd.merge(master_df, new_ids, on='subject_id', how='left')
    # update the subj_id_mapper with the new ids from the data table
    subj_id_mapper.update(new_ids.set_index('subject_id')['ASAP_sample_id'].to_dict())

# extract the new ids from the subject_df as an updated subj_id_mapper
subj_id_mapper = master_df[['subject_id','ASAP_sample_id']].set_index('subject_id')['ASAP_sample_id'].to_dict()

# update the subj_id_mapper with the new ids from the data table
master_df = pd.merge(master_df, subject_df,
                    left_on=['subject_id'], right_on=['sample_id'], how='inner')

# return the updated id_mapper, updated subject_df, and the starting index for the new ids
return subj_id_mapper, master_df, master_df.shape[0]


# generate new ids for all the subjects in the data table
updated_subj_id_mapper, master_df, n = generate_asap_subject_ids(subj_id_mapper, subject_df)


df_dups = subject_df[subject_df.duplicated(keep=False, subset=['subject_id'])].sort_values('subject_id').reset_index(drop = True).copy()

study_tracker_df = pd.DataFrame()
# study_tracker_df = pd.DataFrame.from_dict(subj_id_mapper,
#                                                         orient='index',
#                                                         columns = ['ASAP_sample_id','subject_id'])\
#                                                     .rename_axis('master_sample_id').reset_index()\
#                                                     .astype(str)
# Check if any sample ID exists in df_subset.
sample_id_unique = pd.merge(study_tracker_df, subject_df,
                            left_on=['subject_id'], right_on=['sample_id'], how='inner')

In [22]:
# extract the max value of the mapper's third (last) section ([2] or [-1]) to get our n
if bool(subj_id_mapper):
    n = max([int(v.split("_")[2]) for v in subj_id_mapper.values() if v]) + 1
else:
    n = 1
nstart = n

df_nodups_wids = subject_df.copy()
# might want to use 'source_subject_id' instead of 'subject_id' since we want to find matches across teams
# shouldn't actually matter but logically cleaner
uids = [str(id) for id in df_nodups_wids['subject_id'].unique()]
mapid = {}
for uid in uids:
    mapid[uid]= n
    n += 1

df_nodups_wids['uid_idx'] = df_nodups_wids['subject_id'].map(mapid)
# make a new column with the ASAP_subject_id
# and insert it at the beginning of the dataframe
ASAP_subject_id = [f'{STUDY_PREFIX}{i:06}' for i in df_nodups_wids.uid_idx]
df_nodups_wids.insert(0, 'ASAP_subject_id', ASAP_subject_id)

df_nodups_wids['uid_idx_cumcount'] = df_nodups_wids.groupby('ASAP_subject_id').cumcount() + 1
asap_id_mapper = dict(zip(df_nodups_wids['subject_id'], df_nodups_wids['ASAP_subject_id']))

# update the subj_id_mapper
subj_id_mapper.update(asap_id_mapper)

In [20]:
if df_dups.shape[0]>0:
    dupids_mapper = dict(zip(df_dups.subject_id.unique(),
                        [num+n for num in range(len(df_dups.subject_id.unique()))]))
    
    df_dup_chunks = []
    for subj_id, samp_n in dupids_mapper.items():
        df_dups_subset = df_dups[df_dups.subject_id==subj_id].copy()
        
        asap_id = dupids_mapper[subj_id]
        df_dups_subset['asap_sample'] = [f'{asap_id}_{samp_n:06}' for i in range(df_dups_subset.shape[0])]
        df_dups_subset['samp_rep_no'] = ['s'+str(i+1) for i in range(df_dups_subset.shape[0])]
        # make a new column with the asap_sample_id
        # and insert it at the beginning of the dataframe
        df_dups_subset['ASAP_sample_id'] = df_dups_subset['asap_sample'] + '_' + df_dups_subset['samp_rep_no']

        df_dup_chunks.append(df_dups_subset)
    df_dups_wids = pd.concat(df_dup_chunks)

df_nodups = subject_df[~subject_df.duplicated(keep=False, subset=['subject_id'])].sort_values('subject_id').reset_index(drop = True).copy()


In [21]:
df_dups_wids

,subject_id,source_subject_id,AMPPD_id,GP2_id,biobank_name,organism,sex,age_at_collection,race,ethnicity,...,smoking_status,smoking_years,APOE_e4_status,cognitive_status,time_from_baseline,primary_diagnosis,primary_diagnosis_text,asap_sample,samp_rep_no,ASAP_sample_id
0,bovon,P95/10,NaN,MDGAP-QSBB_000046_s1,QSBB_UK,Human,Male,81,NaN,NaN,...,NaN,NaN,NaN,NaN,0,Idiopathic PD,clinpath info: PD | NA,2_000002,s1,2_000002_s1
1,bovon,dup_1P95/10,NaN,MDGAP-QSBB_000046_s1,QSBB_UK,Human,Male,81,NaN,NaN,...,NaN,NaN,NaN,NaN,0,Idiopathic PD,clinpath info: PD | NA,2_000002,s2,2_000002_s2
2,himuv,P82/12,NaN,MDGAP-QSBB_000593_s1,QSBB_UK,Human,Male,69,NaN,NaN,...,NaN,NaN,NaN,NaN,0,Idiopathic PD,clinpath info: PDD | Atypical Parkinson's and ...,3_000003,s1,3_000003_s1
3,himuv,dup_1P82/12,NaN,MDGAP-QSBB_000593_s1,QSBB_UK,Human,Male,69,NaN,NaN,...,NaN,NaN,NaN,NaN,0,Idiopathic PD,clinpath info: PDD | Atypical Parkinson's and ...,3_000003,s2,3_000003_s2
4,jopin,dup_1P67/16,NaN,MDGAP-QSBB_000538_s1,QSBB_UK,Human,Female,68,NaN,NaN,...,NaN,NaN,NaN,NaN,0,Idiopathic PD,clinpath info: PDD | PD with dementia,4_000004,s1,4_000004_s1
5,jopin,P67/16,NaN,MDGAP-QSBB_000538_s1,QSBB_UK,Human,Female,68,NaN,NaN,...,NaN,NaN,NaN,NaN,0,Idiopathic PD,clinpath info: PDD | PD with dementia,4_000004,s2,4_000004_s2
6,juzot,dup_1P13/17,NaN,MDGAP-QSBB_000246_s1,QSBB_UK,Human,Male,83,NaN,NaN,...,NaN,NaN,NaN,NaN,0,Idiopathic PD,clinpath info: PDD | PD (with dementia),5_000005,s1,5_000005_s1
7,juzot,P13/17,NaN,MDGAP-QSBB_000246_s1,QSBB_UK,Human,Male,83,NaN,NaN,...,NaN,NaN,NaN,NaN,0,Idiopathic PD,clinpath info: PDD | PD (with dementia),5_000005,s2,5_000005_s2
8,litan,dup_1P26/11,NaN,MDGAP-QSBB_000325_s1,QSBB_UK,Human,Female,73,NaN,NaN,...,NaN,NaN,NaN,NaN,0,No PD nor other neurological disorder,clinpath info: Pathological ageing | Control,6_000006,s1,6_000006_s1
9,litan,P26/11,NaN,MDGAP-QSBB_000325_s1,QSBB_UK,Human,Female,73,NaN,NaN,...,NaN,NaN,NaN,NaN,0,No PD nor other neurological disorder,clinpath info: Pathological ageing | Control,6_000006,s2,6_000006_s2


In [ ]:


# extract the max value of the mapper's third (last) section ([2] or [-1]) to get our n
if bool(subj_id_mapper):
    n = max([int(v.split("_")[2]) for v in subj_id_mapper.values() if v]) + 1
else:
    n = 1
nstart = n
df_nodups_wids = subject_df.copy()



In [13]:
df_nodups_wids

,subject_id,source_subject_id,AMPPD_id,GP2_id,biobank_name,organism,sex,age_at_collection,race,ethnicity,...,hx_dementia_mci,hx_melanoma,education_level,smoking_status,smoking_years,APOE_e4_status,cognitive_status,time_from_baseline,primary_diagnosis,primary_diagnosis_text
0,jopin,dup_1P67/16,NaN,MDGAP-QSBB_000538_s1,QSBB_UK,Human,Female,68,NaN,NaN,...,Yes,NaN,NaN,NaN,NaN,NaN,NaN,0,Idiopathic PD,clinpath info: PDD | PD with dementia
1,mivog,dup_1P56/08,NaN,MDGAP-QSBB_000030_s1,QSBB_UK,Human,Male,65,NaN,NaN,...,Yes,NaN,NaN,NaN,NaN,NaN,NaN,0,Idiopathic PD,clinpath info: PDD | PD
2,rijof,dup_1P8/18,NaN,MDGAP-QSBB_000583_s1,QSBB_UK,Human,Male,84,NaN,NaN,...,No,NaN,NaN,NaN,NaN,NaN,NaN,0,Idiopathic PD,clinpath info: PD | PD
3,juzot,dup_1P13/17,NaN,MDGAP-QSBB_000246_s1,QSBB_UK,Human,Male,83,NaN,NaN,...,Yes,NaN,NaN,NaN,NaN,NaN,NaN,0,Idiopathic PD,clinpath info: PDD | PD (with dementia)
4,nobin,dup_1P46/10,NaN,MDGAP-QSBB_000422_s1,QSBB_UK,Human,Female,83,NaN,NaN,...,Yes,NaN,NaN,NaN,NaN,NaN,NaN,0,Idiopathic PD,clinpath info: PDD | PD (with dementia)
5,bovon,dup_1P95/10,NaN,MDGAP-QSBB_000046_s1,QSBB_UK,Human,Male,81,NaN,NaN,...,No,NaN,NaN,NaN,NaN,NaN,NaN,0,Idiopathic PD,clinpath info: PD | NA
6,himuv,dup_1P82/12,NaN,MDGAP-QSBB_000593_s1,QSBB_UK,Human,Male,69,NaN,NaN,...,Yes,NaN,NaN,NaN,NaN,NaN,NaN,0,Idiopathic PD,clinpath info: PDD | Atypical Parkinson's and ...
7,zidip,dup_1P30/09,NaN,MDGAP-QSBB_000101_s1,QSBB_UK,Human,Female,77,NaN,NaN,...,No,NaN,NaN,NaN,NaN,NaN,NaN,0,Idiopathic PD,clinpath info: PD | PD
8,nafam,dup_1P17/16,NaN,MDGAP-DEMENTIA_000249_s1,QSBB_UK,Human,Female,84,NaN,NaN,...,No,NaN,NaN,NaN,NaN,NaN,NaN,0,No PD nor other neurological disorder,clinpath info: Control | Control
9,posaj,dup_1P47/11,NaN,MDGAP-QSBB_000427_s1,QSBB_UK,Human,Female,79,NaN,NaN,...,No,NaN,NaN,NaN,NaN,NaN,NaN,0,No PD nor other neurological disorder,clinpath info: Control | Control


In [14]:

# might want to use 'source_subject_id' instead of 'subject_id' since we want to find matches across teams
# shouldn't actually matter but logically cleaner
uids = [str(id) for id in df_nodups_wids['subject_id'].unique()]
mapid = {}
for uid in uids:
    mapid[uid]= n
n += 1


In [15]:
mapid

{'jopin': 1,
 'mivog': 1,
 'rijof': 1,
 'juzot': 1,
 'nobin': 1,
 'bovon': 1,
 'himuv': 1,
 'zidip': 1,
 'nafam': 1,
 'posaj': 1,
 'lutit': 1,
 'rusiz': 1,
 'litan': 1,
 'dup_2jopin': 1,
 'dup_2mivog': 1,
 'dup_2rijof': 1,
 'dup_2juzot': 1,
 'dup_2nobin': 1,
 'dup_2bovon': 1,
 'dup_2himuv': 1,
 'dup_2zidip': 1,
 'dup_2nafam': 1,
 'dup_2posaj': 1,
 'dup_2lutit': 1,
 'dup_2rusiz': 1,
 'dup_2litan': 1}

In [ ]:

df_nodups_wids['uid_idx'] = df_nodups_wids['subject_id'].map(mapid)
# make a new column with the ASAP_subject_id
# and insert it at the beginning of the dataframe
ASAP_subject_id = [f'{STUDY_PREFIX}{i:06}' for i in df_nodups_wids.uid_idx]
df_nodups_wids.insert(0, 'ASAP_subject_id', ASAP_subject_id)

df_nodups_wids['uid_idx_cumcount'] = df_nodups_wids.groupby('ASAP_subject_id').cumcount() + 1
asap_id_mapper = dict(zip(df_nodups_wids['subject_id'], df_nodups_wids['ASAP_subject_id']))

# update the subj_id_mapper
subj_id_mapper.update(asap_id_mapper)

# return subj_id_mapper, df_nodups_wids, nstart


In [38]:
subj_id_mapper

{'HC_1225': 'ASAP_PMBDS_000001',
 'HC_0602': 'ASAP_PMBDS_000002',
 'PD_0009': 'ASAP_PMBDS_000003',
 'PD_1921': 'ASAP_PMBDS_000004',
 'PD_2058': 'ASAP_PMBDS_000005',
 'PD_1441': 'ASAP_PMBDS_000006',
 'PD_1344': 'ASAP_PMBDS_000007',
 'HC_1939': 'ASAP_PMBDS_000008',
 'HC_1308': 'ASAP_PMBDS_000009',
 'HC_1862': 'ASAP_PMBDS_000010',
 'HC_1864': 'ASAP_PMBDS_000011',
 'HC_2057': 'ASAP_PMBDS_000012',
 'HC_2061': 'ASAP_PMBDS_000013',
 'HC_2062': 'ASAP_PMBDS_000014',
 'HC_2067': 'ASAP_PMBDS_000015',
 'PD_0348': 'ASAP_PMBDS_000016',
 'PD_0413': 'ASAP_PMBDS_000017',
 'PD_1312': 'ASAP_PMBDS_000018',
 'PD_1317': 'ASAP_PMBDS_000019',
 'PD_1504': 'ASAP_PMBDS_000020',
 'PD_1858': 'ASAP_PMBDS_000021',
 'PD_1902': 'ASAP_PMBDS_000022',
 'PD_1973': 'ASAP_PMBDS_000023',
 'PD_2005': 'ASAP_PMBDS_000024',
 'PD_2038': 'ASAP_PMBDS_000025'}

In [39]:
# since the current SAMPLE tables can have multipl sample_ids lets drop duplciates, with the caveat of replciates
df_nodups = sample_df.drop_duplicates(subset=['sample_id'])

# 
uniq_subj = df_nodups.subject_id.unique()

dupids_mapper = dict(zip(uniq_subj,
                    [num+nstart for num in range(len(uniq_subj))] ))
dupids_mapper

{'HC_1225': 1,
 'HC_0602': 2,
 'PD_0009': 3,
 'PD_1921': 4,
 'PD_2058': 5,
 'PD_1441': 6,
 'PD_1344': 7,
 'HC_1939': 8,
 'HC_1308': 9,
 'HC_1862': 10,
 'HC_1864': 11,
 'HC_2057': 12,
 'HC_2061': 13,
 'HC_2062': 14,
 'HC_2067': 15,
 'PD_0348': 16,
 'PD_0413': 17,
 'PD_1312': 18,
 'PD_1317': 19,
 'PD_1504': 20,
 'PD_1858': 21,
 'PD_1902': 22,
 'PD_1973': 23,
 'PD_2005': 24,
 'PD_2038': 25}

In [47]:

dupids_mapper = dict(zip(uniq_subj,
                    [num+nstart for num in range(len(uniq_subj))] ))

df_dup_chunks = []
for subj_id, samp_n in dupids_mapper.items():
    df_dups_subset = df_nodups[df_nodups.subject_id==subj_id].copy()
    asap_id = subj_id_mapper[subj_id]
    df_dups_subset['asap_sample'] = [f'{asap_id}_{i:06}' for i in range(df_dups_subset.shape[0])]
    df_dups_subset['samp_rep_no'] = ['s'+str(i+1) for i in range(df_dups_subset.shape[0])]
    # make a new column with the asap_sample_id
    # and insert it at the beginning of the dataframe
    df_dups_subset['ASAP_sample_id'] = df_dups_subset['asap_sample'] + '_' + df_dups_subset['samp_rep_no']

    df_dup_chunks.append(df_dups_subset)



In [48]:
df_dups_subset.shape[0]

3

In [49]:
df_dups_wids = pd.concat(df_dup_chunks)

id_mapper = dict(zip(df_dups_wids.sample_id,
                    df_dups_wids.ASAP_sample_id))
out_df = sample_df.copy()
ASAP_sample_id = out_df['sample_id'].map(id_mapper)
out_df.insert(0, 'ASAP_sample_id', ASAP_sample_id)

samp_id_mapper.update(id_mapper)

samp_id_mapper


{'MFG_HC_1225': 'ASAP_PMBDS_000001_s1',
 'HIP_HC_1225': 'ASAP_PMBDS_000001_s2',
 'SN_HC_1225': 'ASAP_PMBDS_000001_s3',
 'MFG_HC_0602': 'ASAP_PMBDS_000002_s1',
 'HIP_HC_0602': 'ASAP_PMBDS_000002_s2',
 'SN_HC_0602': 'ASAP_PMBDS_000002_s3',
 'MFG_PD_0009': 'ASAP_PMBDS_000003_s1',
 'HIP_PD_0009': 'ASAP_PMBDS_000003_s2',
 'SN_PD_0009': 'ASAP_PMBDS_000003_s3',
 'MFG_PD_1921': 'ASAP_PMBDS_000004_s1',
 'HIP_PD_1921': 'ASAP_PMBDS_000004_s2',
 'SN_PD_1921': 'ASAP_PMBDS_000004_s3',
 'MFG_PD_2058': 'ASAP_PMBDS_000005_s1',
 'HIP_PD_2058': 'ASAP_PMBDS_000005_s2',
 'SN_PD_2058': 'ASAP_PMBDS_000005_s3',
 'MFG_PD_1441': 'ASAP_PMBDS_000006_s1',
 'HIP_PD_1441': 'ASAP_PMBDS_000006_s2',
 'SN_PD_1441': 'ASAP_PMBDS_000006_s3',
 'MFG_PD_1344': 'ASAP_PMBDS_000007_s1',
 'HIP_PD_1344': 'ASAP_PMBDS_000007_s2',
 'SN_PD_1344': 'ASAP_PMBDS_000007_s3',
 'MFG_HC_1939': 'ASAP_PMBDS_000008_s1',
 'HIP_HC_1939': 'ASAP_PMBDS_000008_s2',
 'SN_HC_1939': 'ASAP_PMBDS_000008_s3',
 'MFG_HC_1308': 'ASAP_PMBDS_000009_s1',
 'HIP_HC

In [31]:
SAMPLE.columns

Index(['sample_id', 'subject_id', 'source_sample_id', 'replicate',
       'replicate_count', 'repeated_sample', 'batch', 'tissue', 'brain_region',
       'hemisphere', 'region_level_1', 'region_level_2', 'region_level_3',
       'RIN', 'source_RIN', 'molecular_source', 'input_cell_count', 'assay',
       'sequencing_end', 'sequencing_length', 'sequencing_instrument',
       'organism_ontology_term_id', 'development_stage_ontology_term_id',
       'sex_ontology_term_id', 'self_reported_ethnicity_ontology_term_id',
       'disease_ontology_term_id', 'tissue_ontology_term_id',
       'cell_type_ontology_term_id', 'assay_ontology_term_id',
       'suspension_type', 'DV200', 'pm_PH', 'donor_id'],
      dtype='object')

In [19]:
sample_df = SAMPLE.copy()
if True:

# def generate_asap_sample_ids(subj_id_mapper:dict, 
#                              sample_df:pd.DataFrame, 
#                              nstart:int, 
#                              samp_id_mapper:dict) -> tuple[dict, pd.DataFrame]:
    """
    generate new unique_ids for new sample_ids in sample_df table, 
    update the id_mapper with the new ids from the data table


    return the updated id_mapper and updated sample_df
    """
    # could pass subj_id_mapper as a parameter instead of n.  e.g.
    # if bool(subj_id_mapper):
    #     n = max([int(v.split("_")[2]) for v in subj_id_mapper.values() if v]) + 1
    # else:
    #     n = 1
    
    # since the current SAMPLE tables can have multipl sample_ids lets drop duplciates, with the caveat of replciates
    df_nodups = sample_df.drop_duplicates(subset=['sample_id'])
    
    # 
    uniq_subj = df_nodups.subject_id.unique()

    dupids_mapper = dict(zip(uniq_subj,
                        [num+nstart for num in range(len(uniq_subj))] ))

    df_dup_chunks = []
    for subj_id, samp_n in dupids_mapper.items():
        df_dups_subset = df_nodups[df_nodups.subject_id==subj_id].copy()
        asap_id = subj_id_mapper[subj_id]
        df_dups_subset['asap_sample'] = [f'{asap_id}_{samp_n:06}' for i in range(df_dups_subset.shape[0])]
        df_dups_subset['samp_rep_no'] = ['s'+str(i+1) for i in range(df_dups_subset.shape[0])]
        # make a new column with the asap_sample_id
        # and insert it at the beginning of the dataframe
        df_dups_subset['ASAP_sample_id'] = df_dups_subset['asap_sample'] + '_' + df_dups_subset['samp_rep_no']

        df_dup_chunks.append(df_dups_subset)
    df_dups_wids = pd.concat(df_dup_chunks)

    id_mapper = dict(zip(df_dups_wids.sample_id,
                        df_dups_wids.ASAP_sample_id))
    out_df = sample_df.copy()
    ASAP_sample_id = out_df['sample_id'].map(id_mapper)
    out_df.insert(0, 'ASAP_sample_id', ASAP_sample_id)

    samp_id_mapper.update(id_mapper)

    # return samp_id_mapper, out_df



In [20]:
subj_id_mapper, samp_id_mapper



({'HC_1225': 'ASAP_PMBDS_000001',
  'HC_0602': 'ASAP_PMBDS_000002',
  'PD_0009': 'ASAP_PMBDS_000003',
  'PD_1921': 'ASAP_PMBDS_000004',
  'PD_2058': 'ASAP_PMBDS_000005',
  'PD_1441': 'ASAP_PMBDS_000006',
  'PD_1344': 'ASAP_PMBDS_000007',
  'HC_1939': 'ASAP_PMBDS_000008',
  'HC_1308': 'ASAP_PMBDS_000009',
  'HC_1862': 'ASAP_PMBDS_000010',
  'HC_1864': 'ASAP_PMBDS_000011',
  'HC_2057': 'ASAP_PMBDS_000012',
  'HC_2061': 'ASAP_PMBDS_000013',
  'HC_2062': 'ASAP_PMBDS_000014',
  'HC_2067': 'ASAP_PMBDS_000015',
  'PD_0348': 'ASAP_PMBDS_000016',
  'PD_0413': 'ASAP_PMBDS_000017',
  'PD_1312': 'ASAP_PMBDS_000018',
  'PD_1317': 'ASAP_PMBDS_000019',
  'PD_1504': 'ASAP_PMBDS_000020',
  'PD_1858': 'ASAP_PMBDS_000021',
  'PD_1902': 'ASAP_PMBDS_000022',
  'PD_1973': 'ASAP_PMBDS_000023',
  'PD_2005': 'ASAP_PMBDS_000024',
  'PD_2038': 'ASAP_PMBDS_000025'},
 {'MFG_HC_1225': 'ASAP_PMBDS_000001_000001_s1',
  'HIP_HC_1225': 'ASAP_PMBDS_000001_000001_s2',
  'SN_HC_1225': 'ASAP_PMBDS_000001_000001_s3',
  'MFG

Use the `process_meta_files` function to generate the mappers and update the meta tables.

In [27]:
subject_mapper_path = Path.cwd() / "ASAP_subj_test2.json"
sample_mapper_path = Path.cwd() / "ASAP_samp_test2.json"



export_root = Path.cwd() / "ASAP_tables" 

table_root = Path.cwd() / "clean/team-Lee/v2_20231109/"
## add team Lee
process_meta_files(table_root, 
                       CDE_path, 
                       subject_mapper_path, 
                       sample_mapper_path, 
                       export_path=export_root)



id_mapper not found at /Users/ergonyc/Projects/ASAP/meta-clean/ASAP_subj_test2.json
id_mapper not found at /Users/ergonyc/Projects/ASAP/meta-clean/ASAP_samp_test2.json
exporting to /Users/ergonyc/Projects/ASAP/meta-clean/ASAP_tables/TEAM-LEE


1

Using the same mapper_paths we can continue to generate ASAP IDs

In [28]:

## add team Hafler
table_root = Path.cwd() / "clean/team-Hafler"
table_root = Path.cwd() / "clean/team-Hafler/v2_20231109/"

process_meta_files(table_root, 
                       CDE_path, 
                       subject_mapper_path, 
                       sample_mapper_path, 
                       export_path=export_root)

## add team Hardy
table_root = Path.cwd() / "clean/team-Hardy/v2_20231109/"
process_meta_files(table_root, 
                       CDE_path, 
                       subject_mapper_path, 
                       sample_mapper_path, 
                       export_path=export_root)


id_mapper loaded from /Users/ergonyc/Projects/ASAP/meta-clean/ASAP_subj_test2.json
id_mapper loaded from /Users/ergonyc/Projects/ASAP/meta-clean/ASAP_samp_test2.json
exporting to /Users/ergonyc/Projects/ASAP/meta-clean/ASAP_tables/TEAM-HAFLER
id_mapper loaded from /Users/ergonyc/Projects/ASAP/meta-clean/ASAP_subj_test2.json
id_mapper loaded from /Users/ergonyc/Projects/ASAP/meta-clean/ASAP_samp_test2.json
exporting to /Users/ergonyc/Projects/ASAP/meta-clean/ASAP_tables/TEAM-HARDY


1

In [30]:
Path(".")/ "clean/team-Hardy/v2_20231109/"

PosixPath('clean/team-Hardy/v2_20231109')